In [1]:
from sklearn.externals import joblib
import pandas as pd
import os
import sys
sys.path.append('../src')
from transforms import *

DATA_FOLDER = '../data/csv'

DATA = {'FOURIER': 'v000_SCIG_SC_SENSORA_FOURIER_chunk_90', 
        'HOS': 'v000_SCIG_SC_SENSORA_HOS_chunk_90', 
        'SCM': 'v000_SCIG_SC_SENSORA_SCM_chunk_90'}

LABELS = {'FOURIER': 'v000_SCIG_SC_SENSORA_FOURIER_labels_chunk_90', 
          'HOS': 'v000_SCIG_SC_SENSORA_HOS_labels_chunk_90', 
          'SCM': 'v000_SCIG_SC_SENSORA_SCM_labels_chunk_90'}

SEED = 6969

/home/navar/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/navar/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load data

In [2]:
feature_set = 'FOURIER'

data = pd.read_csv(os.path.join(DATA_FOLDER, DATA[feature_set]) + '.csv')
labels = pd.read_csv(os.path.join(DATA_FOLDER, LABELS[feature_set]) + '.csv')

In [3]:
data.head()

,idx,fx1_R,fx0d5_R,fx1d5_R,fx2d5_R,fx3_R,fx5_R,fx7_R,Freq_Rated,Freq_Gen,CC_bus,Power,I_R_rms,I_S_rms,I_T_rms,Load,Class
0,106.0,0.984892,0.023496,0.033428,0.021333,0.201807,0.371382,0.319380,52.5,51.63,287.0,0.24,1.879780,1.661605,1.775258,0.0,4.0
1,24.0,0.867996,0.014729,0.038349,0.018842,0.150441,0.384602,0.275356,45.0,44.25,295.0,0.26,1.871471,1.764580,1.851097,0.0,3.0
2,36.0,1.218724,0.020471,0.031999,0.015327,0.143213,0.390037,0.398740,47.5,46.31,356.0,0.50,3.016166,2.654177,2.725753,0.0,6.0
3,53.0,0.855593,0.016479,0.043882,0.009739,0.142357,0.354431,0.327678,47.5,46.74,284.0,0.23,1.779976,1.672404,1.735441,0.0,3.0
4,83.0,1.171638,0.115717,0.053422,0.066587,0.152327,0.386218,0.377228,52.5,51.48,360.0,0.48,2.687841,2.669896,2.756524,0.0,2.0


In [10]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('cleaner', DropNaN()),
                     ('selector', FeatureSelection(extractor=feature_set)),
                     ('scaler', FeatureScaling()),
                    ])

In [12]:
features = pipeline.fit_transform(data)
features.head()

,fx0d5_R,fx1d5_R,fx2d5_R,fx3_R,fx5_R,fx7_R,Freq_Gen,CC_bus
0,-0.167465,-0.295381,-0.068850,1.023716,-0.398822,-0.768443,-0.008417,-0.698328
1,-0.515840,-0.079413,-0.206292,-0.832800,-0.223521,-1.468638,-1.478117,-0.511118
2,-0.287671,-0.358134,-0.400194,-1.094025,-0.151464,0.493779,-1.067875,0.916361
3,-0.446329,0.163424,-0.708516,-1.124984,-0.623582,-0.636464,-0.982242,-0.768532
4,3.497504,0.582133,2.427985,-0.764607,-0.202099,0.151642,-0.038289,1.009966


In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
clf_mlp = MLPClassifier(activation='tanh', hidden_layer_sizes=(100,), learning_rate='adaptive', tol=1e-6, max_iter=2000)
clf_svm = SVC()
clf_knn = KNeighborsClassifier()
clf_naive_bayes = GaussianNB()

In [14]:
from sklearn.model_selection import cross_val_score

X = features.values
y = labels.values.reshape(-1)

cross_val_score(clf_mlp, X, y, cv=2, scoring='accuracy')

/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


array([0.28714524, 0.30185497])

In [15]:
clf_mlp.fit(X, y)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=1e-06, validation_fraction=0.1,
       verbose=False, warm_start=False)

# Load Data

In [17]:
DATA = 'v000_SCIG_SC_SENSORA_FOURIER_chunk_10.csv'
LABELS = 'v000_SCIG_SC_SENSORA_FOURIER_labels_chunk_10.csv'

data = pd.read_csv(os.path.join(DATA_FOLDER, DATA))
labels = pd.read_csv(os.path.join(DATA_FOLDER, LABELS))

features = pipeline.transform(data)

X_test = features.values
y_test = labels.values.reshape(-1)

In [19]:
y_hat = clf_mlp.predict(X_test)

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_hat))
print(accuracy_score(y_test, y_hat))

[[10  2  1  5  3  2  2]
 [ 4  4  3  2  3  4  0]
 [ 0  4  3  2  1  8  0]
 [ 4  1  1  7  1  0  4]
 [ 6  2  3  3  1  3  0]
 [ 3  2  3  1  4  8  0]
 [ 1  0  0  7  1  0  4]]
0.2781954887218045
